In [48]:
from policyengine_us import Simulation
from policyengine_core.reforms import Reform
import numpy as np
import plotly.graph_objects as go
from policyengine_core.charts import *

In [49]:
reform = Reform.from_dict({
  "gov.states.mi.tax.income.rate": {
    "2025-01-01.2100-12-31": 0.0405
  }
}, country_id="us")

In [50]:
situation = {
  "people": {
    "you": {
      "age": {
        "2025": 40
      }
    },
    "your partner": {
      "age": {
        "2025": 40
      }
    }
  },
  "families": {
    "your family": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "spm_units": {
    "your household": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "tax_units": {
    "your tax unit": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "households": {
    "your household": {
      "members": [
        "you",
        "your partner"
      ],
      "state_name": {
        "2025": "MI"
      }
    }
  },
  "marital_units": {
    "your marital unit": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "axes": [
    [
      {
        "name": "employment_income",
        "count": 50,
        "min": 0,
        "max": 50000
      }
    ]
  ]
}


In [51]:
simulation = Simulation(
    reform=reform,
    situation=situation,
)

baseline_simulation = Simulation(
    situation=situation,
)


In [52]:
mi_eitc = simulation.calculate("mi_eitc", 2025)
mi_home_heating_credit = simulation.calculate("mi_home_heating_credit", 2025)

In [53]:
baseline_tax = baseline_simulation.calculate("mi_income_tax", 2025)
reform_tax = simulation.calculate("mi_income_tax", 2025)
difference_tax = reform_tax - baseline_tax

In [59]:
mi_tax_before_credits = simulation.calculate("mi_income_tax_before_refundable_credits", 2025)
baseline_tax_before_credits = baseline_simulation.calculate("mi_income_tax_before_refundable_credits", 2025)

In [78]:
def create_mi_tax_components_graph():
    # Define colors from the Minnesota notebook
    colors = {
        "single_wfc": "#4d94ff",  # Light blue
        "married_wfc": "#0066cc",  # Dark blue
        "single_child": "#999999",  # Light grey
        "married_child": "#333333",  # Dark grey
    }
    
    x = np.linspace(0, 200000, 200)
    
    fig = go.Figure()
    
    # Add EITC trace with dark blue color
    fig.add_trace(
        go.Scatter(
            x=x,
            y=mi_eitc,
            mode="lines",
            name="Michigan EITC",
            line=dict(color=colors["married_wfc"], width=2),
        )
    )
    
    # Add Home Heating Credit trace with light blue color
    fig.add_trace(
        go.Scatter(
            x=x,
            y=mi_home_heating_credit,
            mode="lines",
            name="Michigan Home Heating Credit",
            line=dict(color=colors["single_wfc"], width=2),
        )
    )
    
    # Add Income Tax Change trace with dark grey color
    fig.add_trace(
        go.Scatter(
            x=x,
            y=mi_tax_before_credits,
            mode="lines",
            name="Income Tax before Refundable Credits with Tax Cut",
            line=dict(color=colors["married_child"], width=2),
        )
    )
    
    # Add baseline tax before credits as a dotted line
    fig.add_trace(
        go.Scatter(
            x=x,
            y=baseline_tax_before_credits,
            mode="lines",
            name="Income Tax before Credits without Tax Cut",
            line=dict(color=colors["single_child"], width=2, dash="dash"),
        )
    )
    
    fig.update_layout(
        xaxis_title="Head Earnings",
        yaxis_title="Amount ($)",
        xaxis=dict(tickformat="$,.0f", range=[0, 30000]),
        yaxis=dict(tickformat="$,.0f"),
        # Place legend above the chart
        legend=dict(
            orientation="h",  # Horizontal orientation
            yanchor="bottom",
            y=1.02,  # Position slightly above the plot area
            x=0.01,  # Center horizontally
        ),
        height=600,
        width=900,
    )
    
    return fig

In [79]:
# Create and display the chart
fig = create_mi_tax_components_graph()
fig = format_fig(fig)
fig.show()